In [1]:
import os

In [4]:
from langchain.llms import OpenAI                         # To call OpenAI API
from langchain_groq import ChatGroq              # To call OpenAI Chat API
from langchain.prompts import PromptTemplate              # To create prompt templates
from langchain.chains import LLMChain                     # To create chains of calls to LLMs
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback       # To track token usage
import os
import json
import pandas as pd
import traceback                                          # For error handling
from dotenv import load_dotenv                            # For storing environment variables in local workspace
import PyPDF2

In [ ]:
load_dotenv()  # Load environment variables from .env file

True

In [6]:
key=os.getenv("GROQ_API_KEY")

In [7]:
print(key)

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


In [29]:
llm = ChatGroq(
    groq_api_key=key,
    model_name="llama-3.1-8b-instant",   # or "mixtral-8x7b-32768"
    temperature=0.7
)


In [30]:
llm    # Initialize the LLM with your OpenAI API key

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000188544A7790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000188544A4650>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [31]:
with open("Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [32]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [63]:
# TEMPLATE="""
# Text:{text}
# You are an expert MCQ maker. Given the above text, it is your job to \
# create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
# Make sure the questions are not repeated and check all the questions to be conforming the text as well.
# Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
# Ensure to make {number} MCQs
# ### RESPONSE_JSON
# {RESPONSE_JSON}

# """

TEMPLATE = """
Text: {text}

You are an expert MCQ maker. 
Create exactly {number} multiple choice questions for {subject} students in {tone} tone. 

⚠️ Rules:
- Output MUST be valid JSON.
- Follow the structure in RESPONSE_JSON exactly.
- Do not add Python-style dicts, only JSON (keys/strings in double quotes).
- Ensure all {number} questions are unique and relevant to the text.

### RESPONSE_JSON (example format)
{RESPONSE_JSON}
"""

In [64]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [65]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

In [66]:
# TEMPLATE2="""
# You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
# You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
# if the quiz is not at per with the cognitive and analytical abilities of the students,\
# update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
# Quiz_MCQs:
# {quiz}

# Check from an expert English Writer of the above quiz:
# """

TEMPLATE2 = """
You are an expert English grammarian and writer. 
Given a Multiple Choice Quiz for {subject} students:

1. Evaluate the complexity of the quiz in **no more than 50 words**. 
2. If the quiz is not appropriate for the students’ cognitive and analytical abilities:
   - Revise only the necessary questions.
   - Keep the tone simple and suitable for the students.
3. Always return the final output in **valid JSON format**, following the structure below:

{RESPONSE_JSON}

Quiz_MCQs:
{quiz}

### Final Output (JSON only):
"""


In [67]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE2
)

In [68]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review",verbose=True)

In [69]:
generate_evaluate_chain = SequentialChain(
	chains=[quiz_chain, review_chain],
	input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
	output_variables=["quiz", "review"],
	verbose=True
)

In [70]:
PATH="data.txt"

In [71]:
with open(PATH, "r") as file:
    TEXT=file.read()

In [72]:
print(TEXT)

Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts. Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs).


In [74]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText: {text}\n\nYou are an expert MCQ maker. \nCreate exactly {number} multiple choice questions for {subject} students in {tone} tone. \n\n⚠️ Rules:\n- Output MUST be valid JSON.\n- Follow the structure in RESPONSE_JSON exactly.\n- Do not add Python-style dicts, only JSON (keys/strings in double quotes).\n- Ensure all {number} questions are unique and relevant to the text.\n\n### RESPONSE_JSON (example format)\n{RESPONSE_JSON}\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000188544A7790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000188544A4650>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), output_key='quiz', output_parser=StrOutputParser(), llm_kwargs={}), LLMChain(

In [75]:
TEXT
NUMBER=5
SUBJECT='GENAI'
TONE='SIMPLE'
RESPONSE_JSON=RESPONSE_JSON

In [76]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [77]:
json.dumps(RESPONSE_JSON)     # To convert Python dictionary to JSON string

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [78]:
response=generate_evaluate_chain(
    {

    "text":TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "RESPONSE_JSON":json.dumps(RESPONSE_JSON, indent=2)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts. Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs).

You are an expert MCQ maker. 
Create exactly 5 multiple choice questions for GENAI students in SIMPLE tone. 

⚠️ Rules:
- Output MUST be valid JSON.
- Follow the structure in RESPONSE_JSON exactly.
- Do not add Python-style dicts, only JSON (keys/strings in double quotes).
- Ensur

In [79]:
response

{'text': 'Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts. Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs).',
 'number': 5,
 'subject': 'GENAI',
 'tone': 'SIMPLE',
 'RESPONSE_JSON': '{\n  "1": {\n    "no": "1",\n    "mcq": "multiple choice questions",\n    "options": {\n      "a": "choice here",\n      "b": "choice here",\n      "c": "choice here",\n      "d": "choice here"\n    },\n    "correct": "correct answer"\n  },\n  "2": {\n    "no": "2",\n    "mcq": "multiple choice questions"

In [80]:
quiz=response.get("quiz")

In [81]:
quiz=json.loads(quiz)

In [82]:
quiz


{'1': {'no': '1',
  'mcq': 'What is Generative AI used for?',
  'options': {'a': 'Data analysis and interpretation',
   'b': 'Production of text, images, videos, or other forms of data',
   'c': 'Deep learning model training',
   'd': 'Natural Language Processing'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': 'What technology made the AI boom in the 2020s possible?',
  'options': {'a': 'Improvements in deep neural networks',
   'b': 'Large language models (LLMs)',
   'c': 'Both a and b',
   'd': 'Neither a nor b'},
  'correct': 'c'},
 '3': {'no': '3',
  'mcq': 'What form does the input often come in for Generative AI?',
  'options': {'a': 'Structured data',
   'b': 'Unstructured data',
   'c': 'Natural language prompts',
   'd': 'Raw data'},
  'correct': 'c'},
 '4': {'no': '4',
  'mcq': 'What do Generative AI models learn from their training data?',
  'options': {'a': 'Patterns and structures',
   'b': 'Random noise',
   'c': 'Irrelevant data',
   'd': 'Only images'},
  'correct': 'a

In [88]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [89]:
quiz_table_data

[{'MCQ': 'What is Generative AI used for?',
  'Choices': 'a: Data analysis and interpretation | b: Production of text, images, videos, or other forms of data | c: Deep learning model training | d: Natural Language Processing',
  'Correct': 'b'},
 {'MCQ': 'What technology made the AI boom in the 2020s possible?',
  'Choices': 'a: Improvements in deep neural networks | b: Large language models (LLMs) | c: Both a and b | d: Neither a nor b',
  'Correct': 'c'},
 {'MCQ': 'What form does the input often come in for Generative AI?',
  'Choices': 'a: Structured data | b: Unstructured data | c: Natural language prompts | d: Raw data',
  'Correct': 'c'},
 {'MCQ': 'What do Generative AI models learn from their training data?',
  'Choices': 'a: Patterns and structures | b: Random noise | c: Irrelevant data | d: Only images',
  'Correct': 'a'},
 {'MCQ': 'When did Generative AI tools become more common?',
  'Choices': 'a: 2010s | b: 2020s | c: Both a and b | d: Neither a nor b',
  'Correct': 'b'}]

In [90]:
quiz=pd.DataFrame(quiz_table_data)

In [91]:
quiz.to_csv("genai.csv", index=False)